In [ ]:
import sempy.fabric as fabric

In [ ]:
import notebookutils
import pandas as pd
import re
import requests
import msal
import json 

In [ ]:
dataverse_prefix = "" # Example: cr8dc

wsId = "" # replace with your workspace ID
dsId = "" # replace with your dataset ID
tenant_id = "" # replace with your tenant ID

## SEMPY

In [ ]:
df_datasets = fabric.list_datasets()
df_datasets

In [ ]:
execution_of_dax = fabric.evaluate_dax(
    workspace = wsId, 
    dataset = dsId, 
    dax_string = """
        EVALUATE 
            ADDCOLUMNS(
                SELECTCOLUMNS(
                    SUMMARIZE(
                        'fact Sales',
                        'dim Employees'[Employee],
                        'dim Pricelists'[Product name],
                        'dim DateKey'[Month]
                    ),
                    "Employee",'dim Employees'[Employee], 
                    "Product name",'dim Pricelists'[Product name], 
                    "Month",'dim DateKey'[Month]
                ),
                "Sold units", CALCULATE(SUM('fact Sales'[Quantity]))
            )
    """
)

execution_of_dax

## REST API

In [ ]:
pbi_resource = "pbi"
pbi_Uri = 'https://api.powerbi.com/v1.0/myorg/'

In [ ]:
def get_token():
    return notebookutils.credentials.getToken(pbi_resource)

In [ ]:
dax_query = """DEFINE
	VAR __DS0FilterTable = 
		FILTER(
			KEEPFILTERS(VALUES('Products calculation'[Products calculation])),
			'Products calculation'[Products calculation] = 3
		)
 
	VAR __DS0FilterTable2 = 
		FILTER(
			KEEPFILTERS(VALUES('Dynamic percentage'[Dynamic percentage])),
			'Dynamic percentage'[Dynamic percentage] = 40
		)
 
	VAR __DS0Core = 
		SUMMARIZECOLUMNS(
			'dim Employees'[Employee],
			__DS0FilterTable,
			__DS0FilterTable2,
			"Dynamic_vs_Total_Revenue_Difference", 'Measure'[# Dynamic vs Total Revenue Difference],
			"Total_Revenue", 'Measure'[# Total Revenue],
			"Dynamic_Revenue", 'Measure'[# Dynamic Revenue],
			"Dynamic_Profit", 'Measure'[# Dynamic Profit],
			"Total_Profit", 'Measure'[# Total Profit],
			"Dynamic_vs_Total_Profit_Difference", 'Measure'[# Dynamic vs Total Profit Difference],
			"Dynamic_Revenue_ratio", 'Measure'[% Dynamic Revenue ratio],
			"Dynamic_Profit_ratio", 'Measure'[% Dynamic Profit ratio]
		)
 
EVALUATE
	__DS0Core
"""

In [ ]:
def post_ExecuteQuery(wsId, dsId, daxQ):
    fullurl = f'{pbi_Uri}/groups/{wsId}/datasets/{dsId}/executeQueries'
    pbi_access_token = get_token()
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {pbi_access_token}'}
    content = {"queries": [{"query": daxQ}],"serializerSettings": {"includeNulls": True}}
    response = requests.post(fullurl, json=content, headers = headers, timeout=60).json()['results'][0]['tables'][0]['rows']
    return response


In [ ]:
def rename_columns_with_brackets(df,prefix):
    def extract_bracket_content(col_name):
        match = re.search(r'\[(.*?)\]', col_name)
        return (prefix+"_"+match.group(1)).lower() if match else col_name

    df.rename(columns=lambda x: extract_bracket_content(x), inplace=True)
    return df

In [ ]:
response = post_ExecuteQuery(wsId,dsId,dax_query)
response = rename_columns_with_brackets(pd.DataFrame(response),dataverse_prefix)
display(response)

In [ ]:
json_output = response.to_json(orient="records")
json_output = json.loads(json_output)

## Write to Dataverse

In [ ]:
keyvault_name = "" # replace with your Key Vault name 
keyvault_ir = f'https://{keyvault_name}.vault.azure.net/'

client_id = notebookutils.credentials.getSecret(keyvault_ir,'dataverseFabricIntegrationClientId')
client_secret = notebookutils.credentials.getSecret(keyvault_ir,'dataverseFabricIntegrationClientSecret')
dataverse_name = "" # Example: org9b35785c
resource_uri = f'https://{dataverse_name}.crm4.dynamics.com'

authority_url = f'https://login.microsoftonline.com/{tenant_id}'
scope = [f'{resource_uri}/.default']

table_name = "" # replace with your table name
entity_name = f'{dataverse_prefix}_{table_name}'
entity_url = f'{resource_uri}/api/data/v9.2/{entity_name}'

In [ ]:
app = msal.ConfidentialClientApplication(
    client_id,
    authority=authority_url,
    client_credential=client_secret
)

token_response = app.acquire_token_for_client(scopes=scope)
access_token = token_response.get('access_token')

In [ ]:
# Data to be written to Dataverse
data = json_output

for row in json_output:
    
    token_response = app.acquire_token_for_client(scopes=scope)
    access_token = token_response.get('access_token')

    # Headers
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'OData-MaxVersion': '4.0',
        'OData-Version': '4.0'
    }

    # Write data to Dataverse
    response = requests.post(entity_url, headers=headers, json=row)

    if response.status_code == 204:
        print("Data written successfully!")
    else:
        print(f"Failed to write data. Status code: {response.status_code}")
        print(response.content)
